In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import keras
from keras import Model
from tensorflow.keras.layers import Flatten,LSTM,Dense,Flatten,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras_preprocessing.text import Tokenizer
from keras.initializers import glorot_uniform
from sklearn import model_selection

In [ ]:
with open('train.csv', 'r') as file:
  text = file.readlines()

FileNotFoundError: ignored

In [ ]:
x_train = pd.DataFrame()

In [ ]:
word = []
label = []
for n in text:
  n = n.split(",")
  if n[0] == '"2"':
    label.append(1)
  else:
    label.append(0)
  word.append("".join(n[1:]))
x_train['consumer_review'] = word
x_train['polarity_label'] = label

16847

In [ ]:
_, x_set,_,y_set = model_selection.train_test_split(x_train['consumer_review'], x_train['polarity_label'], test_size = 0.2)

**Cleaning the data:**

In [ ]:
def data_prep(in_tex):
  #remove punctuation
  out_tex = re.sub('[^a-zA-Z]', ' ', in_tex)
  #convert upper case to lower case
  out_tex = "".join(list(map(lambda x:x.lower(), out_tex)))
  #remove single characters
  out_tex = re.sub(r"\s+[a-aZ-Z]\s+", ' ', out_tex)

  return out_tex

In [ ]:
text_set = []
for reviews in list(x_set):
  text_set.append(data_prep(reviews))

In [ ]:
x_train = pd.DataFrame()
x_train['consumer_review'] = text_set
x_train['polarity_label'] = list(y_set)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_train["consumer_review"], x_train["polarity_label"], test_size = 0.3)

In [ ]:
x_train = np.array(x_train.values.tolist())
x_test = np.array(x_test.values.tolist())
y_train = np.array(y_train.values.tolist())

**Applying the Tokenizer:**

In [ ]:
#Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
total_size = len(word_index)+1

In [ ]:
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
max_lenght = 100
x_train = pad_sequences(x_train, padding='post', maxlen=max_lenght)
x_test = pad_sequences(x_test, padding='post', maxlen=max_lenght)

**Structuring the model:**

In [ ]:
model = Sequential()
model.add(Embedding(total_size, 20, input_length = max_lenght))
model.add(LSTM(32, dropout = 0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = "sigmoid"))

**Compiling the model:**

In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 20)           290740    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                6784      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 297,557
Trainable params: 297,557
Non-trainable params: 0
_________________________________________________________________


**Training the model:**

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
19/19 [==============================] - 3s 147ms/step - loss: 0.1885 - acc: 0.9474 - val_loss: 0.4959 - val_acc: 0.8002
Epoch 2/10
19/19 [==============================] - 3s 136ms/step - loss: 0.1068 - acc: 0.9724 - val_loss: 0.5720 - val_acc: 0.8121
Epoch 3/10
19/19 [==============================] - 3s 135ms/step - loss: 0.0674 - acc: 0.9852 - val_loss: 0.6753 - val_acc: 0.8022
Epoch 4/10
19/19 [==============================] - 3s 135ms/step - loss: 0.0512 - acc: 0.9894 - val_loss: 0.7593 - val_acc: 0.8071
Epoch 5/10
19/19 [==============================] - 3s 133ms/step - loss: 0.0519 - acc: 0.9903 - val_loss: 0.6456 - val_acc: 0.8051
Epoch 6/10
19/19 [==============================] - 3s 138ms/step - loss: 0.0598 - acc: 0.9890 - val_loss: 0.8074 - val_acc: 0.7824
Epoch 7/10
19/19 [==============================] - 3s 135ms/step - loss: 0.0650 - acc: 0.9877 - val_loss: 0.4987 - val_acc: 0.7953
Epoch 8/10
19/19 [==============================] - 3s 133ms/step - loss: 0.

In [ ]:
model.save("LSTM.h5")